In [1]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torchvision import datasets,transforms
from torch.optim import lr_scheduler
import torchvision.models as models
import matplotlib.pyplot as plt
import time
import os
import copy
from torchsummary import summary
from tqdm import tqdm
import numpy as np

In [5]:
#data_dir = "D:\\RESEARCH\\ImageResearchData\\224Frames\\"

In [6]:
device = torch.device("cuda")

In [0]:
data_dir = "gdrive/My Drive/PreProcessedDataPNP(Images-224*224)/224Frames/"

In [7]:
device= torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [0]:
from google.colab import drive

In [0]:
drive.mount("gdrive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at gdrive


In [5]:
def imshow(img,title = None):
    img = img.numpy().transpose((1,2,0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    img = std*img +mean
    img = np.clip(img,0,1)
    plt.imshow(img)
    
    if title is not None:
        plt.title(title)
    plt.pause(0.001)    

In [6]:
#out = torchvision.utils.make_grid(inputs)
#print(out.shape)
#imshow(out,title=[class_names[x] for x in classes])

In [8]:
class Network(nn.Module):
    def __init__(self):
        super(Network,self).__init__()
        
        self.resnet = models.resnet18(pretrained = True)
        k = 1
        for param in self.resnet.parameters():
            k+=1
            if k<56:
                param.requires_grad = False
        
        self.dense1 = nn.Linear(in_features =1000,out_features = 512)
        self.relu = nn.ReLU(inplace = True)
        self.dense2 = nn.Linear(in_features = 512,out_features = 256)
        self.dense3 = nn.Linear(in_features = 256,out_features = 64)
        self.out = nn.Linear(in_features = 64,out_features = 2)
        
    def forward(self,x):
        x = self.resnet(x)
        x = self.relu(x)
        x = self.dense1(x)
        x = self.relu(x)
        x = self.dense2(x)
        x = self.relu(x)
        x = self.dense3(x)
        x = self.relu(x)
        x = self.out(x)
        return F.softmax(x)

In [9]:
net = Network().to(device)

In [10]:
class pnp:
    def __init__(self,data_dir):
        self.transforms = {
            "train" : transforms.Compose([
                transforms.Resize(224),
                transforms.RandomHorizontalFlip(),
                transforms.ColorJitter(),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ]) ,
            "val":transforms.Compose([
                transforms.Resize(224),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ]),
            "test" : transforms.Compose([
                transforms.Resize(224),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
        }
        self.data_dir  = data_dir
        self.batch_size= 32
        self.images = {x :datasets.ImageFolder(os.path.join(self.data_dir,x),self.transforms[x]) for x in ["train","val","test"] }
        self.dataloader = {x : torch.utils.data.DataLoader(self.images[x],batch_size = self.batch_size,shuffle = True) for x in ["val","train","test"]}
        
    def return_len(self,x):
        return len(self.images[x])

    def return_dataloader(self,x):
        return self.dataloader
    
    def return_batches(self,x):
        for i in range(0,len(self.images[x]),self.batch_size):
            inputs,classes = next(iter(self.dataloader[x]))
            yield(inputs,classes)
        
    def return_classes(self,x):
        return self.images[x].classes
    

In [11]:
data_class = pnp(data_dir)

In [12]:
optimizer = optim.Adam(net.parameters(),lr = 0.0005)
criterion = nn.CrossEntropyLoss().to(device) 

In [13]:
def train(net):
    for i in range(10):
        epoch_loss = 0.0
        epoch_total = 0.0
        epoch_correct= 0.0
        for data in  data_class.return_batches("train"):
            x,y = data[0].to(device),data[1].to(device)
            net.zero_grad()
            y_pred = net(x)
            loss = criterion(y_pred,y)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            y_pred = torch.argmax(y_pred,dim = 1)
            epoch_total += y.shape[0]
            epoch_correct += torch.sum(y_pred==y)
        print("epoch {} : train_loss: {} || train_acc : {}".format(i+1,epoch_loss,np.float(epoch_correct)/np.float(epoch_total)))  
        
        with torch.no_grad():
            total = 0.0
            correct = 0.0
            val_loss = 0.0
            for data in data_class.return_batches("val"):
                x_val,y_val = data[0].to(device),data[1].to(device)
                y_val_p = net(x_val)
                loss = criterion(y_val_p,y_val)
                val_loss  +=loss
                y_val_p = torch.argmax(y_val_p,dim = 1)
                for k in range(y_val.shape[0]):
                    if y_val_p[k]==y_val[k]:
                        correct +=1
                    total+=1
            print("           validation_loss: {} || validation Accuracy :  {}".format(val_loss,np.float(correct)/np.float(total)))
            

In [0]:
train(net)

epoch 1 : train_loss: 132.60393843054771 || train_acc : 0.9517200272479565
           validation_loss: 31.264284133911133 || validation Accuracy :  0.9173259493670886
epoch 2 : train_loss: 133.9510181248188 || train_acc : 0.9463555858310627
           validation_loss: 30.848690032958984 || validation Accuracy :  0.9200949367088608
epoch 3 : train_loss: 129.43081039190292 || train_acc : 0.9592983651226158
           validation_loss: 29.866281509399414 || validation Accuracy :  0.9311708860759493
epoch 4 : train_loss: 129.84943780303 || train_acc : 0.9592983651226158
           validation_loss: 31.01861572265625 || validation Accuracy :  0.9185126582278481
epoch 5 : train_loss: 130.40284532308578 || train_acc : 0.9566587193460491
           validation_loss: 30.461193084716797 || validation Accuracy :  0.9256329113924051
epoch 6 : train_loss: 135.1108240187168 || train_acc : 0.9443119891008175
           validation_loss: 30.604745864868164 || validation Accuracy :  0.9232594936708861
epoc

In [0]:
train(net)

epoch 1 : train_loss: 129.31429016590118 || train_acc : 0.9614271117166212
           validation_loss: 31.050458908081055 || validation Accuracy :  0.9165348101265823
epoch 2 : train_loss: 127.01828247308731 || train_acc : 0.9658549046321526
           validation_loss: 29.268901824951172 || validation Accuracy :  0.9406645569620253
epoch 3 : train_loss: 126.61440345644951 || train_acc : 0.9675579019073569
           validation_loss: 29.609838485717773 || validation Accuracy :  0.9378955696202531
epoch 4 : train_loss: 127.16640928387642 || train_acc : 0.9663658038147139
           validation_loss: 29.6826171875 || validation Accuracy :  0.9347310126582279
epoch 5 : train_loss: 127.4807520210743 || train_acc : 0.9657697547683923
           validation_loss: 30.898324966430664 || validation Accuracy :  0.9216772151898734
epoch 6 : train_loss: 130.52929559350014 || train_acc : 0.9567438692098093
           validation_loss: 30.699899673461914 || validation Accuracy :  0.9204905063291139
epoc

In [0]:
train(net)

epoch 1 : train_loss: 125.38224223256111 || train_acc : 0.971474795640327
           validation_loss: 29.468971252441406 || validation Accuracy :  0.939873417721519
epoch 2 : train_loss: 125.3520385324955 || train_acc : 0.9711341961852861
           validation_loss: 29.114410400390625 || validation Accuracy :  0.9406645569620253
epoch 3 : train_loss: 123.59041905403137 || train_acc : 0.9747104904632152
           validation_loss: 28.626407623291016 || validation Accuracy :  0.9462025316455697
epoch 4 : train_loss: 123.06207612156868 || train_acc : 0.9758174386920981
           validation_loss: 29.4732608795166 || validation Accuracy :  0.9382911392405063
epoch 5 : train_loss: 124.90065389871597 || train_acc : 0.9696866485013624
           validation_loss: 29.61780548095703 || validation Accuracy :  0.932753164556962
epoch 6 : train_loss: 125.06765258312225 || train_acc : 0.9704529972752044
           validation_loss: 29.42667007446289 || validation Accuracy :  0.9367088607594937
epoch 

In [0]:
train(net)

epoch 1 : train_loss: 123.36188143491745 || train_acc : 0.9760728882833788
           validation_loss: 29.36536979675293 || validation Accuracy :  0.9367088607594937
epoch 2 : train_loss: 122.73053684830666 || train_acc : 0.9766689373297003
           validation_loss: 28.85144805908203 || validation Accuracy :  0.9402689873417721
epoch 3 : train_loss: 122.66485720872879 || train_acc : 0.9769243869209809
           validation_loss: 28.791311264038086 || validation Accuracy :  0.9454113924050633
epoch 4 : train_loss: 122.40000665187836 || train_acc : 0.9780313351498637
           validation_loss: 28.842622756958008 || validation Accuracy :  0.9465981012658228
epoch 5 : train_loss: 122.00772535800934 || train_acc : 0.9794788828337875
           validation_loss: 29.057287216186523 || validation Accuracy :  0.9414556962025317
epoch 6 : train_loss: 122.3106010556221 || train_acc : 0.9787125340599455
           validation_loss: 28.07558822631836 || validation Accuracy :  0.9529272151898734
ep

In [0]:
train(net)

epoch 1 : train_loss: 121.18247345089912 || train_acc : 0.9822036784741145
           validation_loss: 28.96276092529297 || validation Accuracy :  0.9465981012658228
epoch 2 : train_loss: 121.61397725343704 || train_acc : 0.9815224795640327
           validation_loss: 29.443256378173828 || validation Accuracy :  0.9371044303797469
epoch 3 : train_loss: 121.56456005573273 || train_acc : 0.9811818801089919
           validation_loss: 28.29401969909668 || validation Accuracy :  0.9517405063291139
epoch 4 : train_loss: 122.3440505862236 || train_acc : 0.9782016348773842
           validation_loss: 28.662797927856445 || validation Accuracy :  0.9473892405063291
epoch 5 : train_loss: 120.8685193657875 || train_acc : 0.9825442779291553
           validation_loss: 29.511648178100586 || validation Accuracy :  0.9339398734177216
epoch 6 : train_loss: 122.08729666471481 || train_acc : 0.9779461852861036
           validation_loss: 29.442386627197266 || validation Accuracy :  0.9371044303797469
ep

In [0]:
dictionary = {"model" : net, "model_dict":net.state_dict(),"optim_dict":optimizer.state_dict(),"epoch":70} 

In [0]:
torch.save(dictionary,"gdrive/My Drive/SupportingFiles/pytorch_image_modelling_70.pt")